# GCC118 - Programação Matemática  
## Universidade Federal de Lavras  

---

## Problema 1  

### Alunos:  
- **Gabriel Jardim de Souza** - 202310530  
- **Paulo Henrique Silveira dos Anjos** - 202310533


## Enunciado:

Uma nutricionista deseja formular uma dieta de emagrecimento, de forma a atender às necessidades mínimas de nutrientes de um indivíduo, respeitando limites calóricos e de custo, e controlando o excesso de alguns nutrientes específicos.

Você dispõe de uma base de dados contendo 30 alimentos, cada um com informações de:

- Preço por porção (R$)
- Valor energético (kcal)
- Proteína (g)
- Gordura (g)
- Carboidrato (g)
- Cálcio (mg)
- Ferro (mg)
- Vitamina C (mg)
- Sódio (mg)

As informações estão armazenadas em um arquivo JSON no seguinte formato simplificado:

```json
{
  "params": {
    "Kmax": 1600,
    "Cmax": 20,
    "min_prot": 60,
    "min_carb": 130,
    "min_cal": 1000,
    "min_fe": 8,
    "min_vitc": 75,
    "fat_max": 50,
    "sod_max": 2300
  },
  "alimentos": [
    { "id": 1, "nome": "Arroz integral", ... },
    { "id": 2, "nome": "Feijão carioca", ... },
    ...
    { "id": 30, "nome": "Sardinha enlatada", ... }
  ]
}
```

### Regras da Dieta

* A dieta diária deve conter no máximo: **1600 kcal R$ 20,00**

* Deve-se garantir um consumo mínimo de:

  - 60 g de proteína

  - 130 g de carboidratos

  - 1000 mg de cálcio

  - 8 mg de ferro

  - 75 mg de vitamina C

* O consumo de gordura deve ser no máximo 50 g/dia

* O consumo de sódio deve ser no máximo 2300 mg/dia

### Objetivo
**Minimizar a quantidade de nutrientes que ultrapassam os limites máximos permitidos.**

## Instalação da biblioteca PuLP

In [67]:
!pip install pulp
import json
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Carregando dados do JSON


In [68]:
with open("dieta.json", "r") as f:
    dados = json.load(f)

params = dados["params"]
alimentos = dados["alimentos"]

## Declaração dos parâmetros

* $kcal\_maximo$: valor máximo de calorias diárias (kcal).

* $custo\_maximo$: valor máximo de custo diário (R$).

* $minimo\_proteina$: consumo mínimo diário de proteína (g).

* $minimo\_carboidrato$: consumo mínimo diário de carboidratos (g).

* $minimo\_calcio$: consumo mínimo diário de cálcio (mg).

* $minimo\_ferro$: consumo mínimo diário de ferro (mg).

* $minimo\_vitaminaC$: consumo mínimo diário de vitamina C (mg).

* $gordura\_maximo$: consumo máximo diário de gordura (g).

* $sodio\_maximo$: consumo máximo diário de sódio (mg).

* $N$: número de alimentos disponíveis.

* $preco$: vetor com o preço de cada alimento (R$).

* $kcal$: vetor com o valor energético de cada alimento (kcal).

* $proteina$: vetor com a quantidade de proteína de cada alimento (g).

* $gordura$: vetor com a quantidade de gordura de cada alimento (g).

* $carboidrato$: vetor com a quantidade de carboidrato de cada alimento (g).

* $calcio$: vetor com a quantidade de cálcio de cada alimento (mg).

* $ferro$: vetor com a quantidade de ferro de cada alimento (mg).

* $vitaminaC$: vetor com a quantidade de vitamina C de cada alimento (mg).

* $sodio$: vetor com a quantidade de sódio de cada alimento (mg).


In [69]:
kcal_maximo = params["Kmax"]
custo_maximo = params["Cmax"]
minimo_proteina = params["min_prot"]
minimo_carboidrato = params["min_carb"]
minimo_calcio = params["min_cal"]
minimo_ferro = params["min_fe"]
minimo_vitaminaC = params["min_vitc"]
gordura_maximo = params["fat_max"]
sodio_maximo = params["sod_max"]

N = len(alimentos)

## Extrai vetor de um nutriente
def vec(key):
    return [alimento.get(key, 0.0) for alimento in alimentos]

preco = vec("preco")
kcal = vec("kcal")
proteina = vec("proteina")
gordura = vec("gordura")
carboidrato = vec("carboidrato")
calcio = vec("calcio")
ferro = vec("ferro")
vitaminaC = vec("vitaminaC")
sodio = vec("sodio")

## Declaração do objeto que representa o modelo matemático

In [70]:
modelo = LpProblem("DietaEmagrecimento", LpMinimize)

## Variáveis de decisão

* $x_i$: número de porções do alimento $i$, com $i = 1, 2, ..., N$, $x_i \geq 0$ e inteiro.

* $excesso\_gordura$: quantidade de gordura que excede o limite máximo permitido na dieta diária, $excesso\_gordura \geq 0$.

* $excesso\_sodio$: quantidade de sódio que excede o limite máximo permitido na dieta diária, $excesso\_sodio \geq 0$.

In [71]:
x = [LpVariable(f"x_{i+1}", lowBound=0, cat='Integer') for i in range(N)]

excesso_gordura = LpVariable("excesso_gordura", lowBound=0)
excesso_sodio  = LpVariable("excesso_sodio", lowBound=0)

## Função objetivo

Minimizar a soma dos excessos de nutrientes:

$\text{min } Z = excesso\_gordura + excesso\_sodio$


In [72]:
modelo += excesso_gordura + excesso_sodio

## Restrições

### Restrições de limite máximo

* Energia (kcal):

  $\sum_{i=1}^{N} kcal_i \cdot x_i \leq kcal\_maximo$

* Custo (R$):

  $\sum_{i=1}^{N} preco_i \cdot x_i \leq custo\_maximo$

* Gordura (g), com variável de excesso:

  $\sum_{i=1}^{N} gordura_i \cdot x_i \leq gordura\_maximo + excesso\_gordura$

* Sódio (mg), com variável de excesso:

  $\sum_{i=1}^{N} sodio_i \cdot x_i \leq sodio\_maximo + excesso\_sodio$

In [73]:
modelo += lpSum(kcal[i] * x[i] for i in range(N)) <= kcal_maximo

modelo += lpSum(preco[i] * x[i] for i in range(N)) <= custo_maximo

modelo += lpSum(gordura[i] * x[i] for i in range(N)) <= gordura_maximo + excesso_gordura

modelo += lpSum(sodio[i]  * x[i] for i in range(N)) <= sodio_maximo + excesso_sodio

### Restrições de consumo mínimo

* Proteína (g):

  $\sum_{i=1}^{N} proteina_i \cdot x_i \geq minimo\_proteina$

* Carboidrato (g):

  $\sum_{i=1}^{N} carboidrato_i \cdot x_i \geq minimo\_carboidrato$

* Cálcio (mg):

  $\sum_{i=1}^{N} calcio_i \cdot x_i \geq minimo\_calcio$

* Ferro (mg):

  $\sum_{i=1}^{N} ferro_i \cdot x_i \geq minimo\_ferro$

* Vitamina C (mg):

  $\sum_{i=1}^{N} vitaminaC_i \cdot x_i \geq minimo\_vitaminaC$

In [74]:
modelo += lpSum(proteina[i] * x[i] for i in range(N)) >= minimo_proteina

modelo += lpSum(carboidrato[i] * x[i] for i in range(N)) >= minimo_carboidrato

modelo += lpSum(calcio[i] * x[i] for i in range(N)) >= minimo_calcio

modelo += lpSum(ferro[i] * x[i] for i in range(N)) >= minimo_ferro

modelo += lpSum(vitaminaC[i] * x[i] for i in range(N)) >= minimo_vitaminaC

### Resolvendo o modelo

In [75]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [76]:
print('status: ', LpStatus[status])
print('funcao objetivo: ', modelo.objective.value())

print("\nExcessos (valores):")
print("  Excesso gordura (g):", excesso_gordura.value())
print("  Excesso sodio (mg):", excesso_sodio.value())

print("\nDieta recomendada: (porções por alimento):")
for i in range(N):
    if x[i].value() > 1e-5:
        print(f"{alimentos[i]['nome']}: {x[i].varValue:.2f} porções ({alimentos[i]['porcao']})")

status:  Optimal
funcao objetivo:  0.0

Excessos (valores):
  Excesso gordura (g): 0.0
  Excesso sodio (mg): 0.0

Dieta recomendada: (porções por alimento):
Pão francês: 3.00 porções (50g)
Iogurte natural: 3.00 porções (100g)
Sardinha enlatada: 2.00 porções (100g)
Alface: 2.00 porções (100g)
Laranja: 1.00 porções (100g)


**Atenção: o problema da dieta pode possuir múltiplas soluções ótimas, mas não há certeza sobre isso.**